# Purpose of this notebook

This notebook is for plotting data from CVs taken on two different areas using SECCM. The scans on different areas should be in different folders, and the separate folders will be plotted using different colours

In [9]:
import matplotlib.pyplot as plt
import numpy as np
import os
import csv
from matplotlib.ticker import AutoMinorLocator
from shapely.geometry import Polygon
from matplotlib.lines import Line2D


%matplotlib qt


def gen_text_from_folder(folder):
    """
    Takes a folder as argument and returns a tuple of (file_names
    , text files) themselves as lists
    """
    data_files = []
    file_names = []
    for root, dirs, files in os.walk(folder):
        for file in files:
            if file.endswith('.txt'):
                if file[-8:-4] == 'info':
                    pass
                else:
                    file_names.append(file)
                    data_files.append(os.path.join(root,file))
    return file_names,data_files

def gen_array_from_text(data_file, file_name):
    """
    Takes a data_file and file_name as arguments as strings
    Returns a (V,I) tuple of potential and current arrays
    """
    data = np.genfromtxt(data_file,skip_header=3)
    data = np.hsplit(data,4)
    V = data[1].flatten()
    I = data[2].flatten()
    return V,I


def convert_cv_to_time(V,I,scan_rate):
    """

    Parameters
    ----------
    V: potential array
    I: current array
    scan_rate: float of scan rate used
    Returns
    -------
    Returns I,T arrays of current and time respectively

    """
    potential_window=V[20]-V[5]
    total_time = abs(potential_window/scan_rate)
    time_between_points = total_time/len(V[5:20])
    T = []
    time = 0
    for i in range(len(I)):
        T.append(time)
        time+=time_between_points

    T = np.array(T)
    return I,T

def array_modifier(V,I,
                double_scan = True,
                smoothen = False,
                normalize = False,
                capacitance_normalize=None,
                custom_normalize = False,
                first_half_only = False,
                tafel_plot = False,
                reverse = False):

        """
        Modifies the arrays in desired ways, returning the (V,I) tuple
        """

        if double_scan == True:
            V = V[512:]
            I = I[512:]

        if smoothen == True:
            I = smooth(I,10)

        if capacitance_normalize != False:
            capacitance = find_capacitance(I,V,reverse=reverse)
            # print(capacitance)
            # scan_label += ", Capacitance = {:.3f} pF".format(capacitance)
            if capacitance != 0:
                
                I = I / capacitance
                if reverse == False:
                    shift = np.mean(I[5:50])
                else:
                    shift = np.mean(I[200:250])

                I = I - shift
            
        if normalize == True:
            shift = np.mean(I[5:50])
            I = I - shift
            I = normalize_func(I)
        
        if custom_normalize != None:
            I = I/custom_normalize

        
        if first_half_only == True:
            V = V[:len(V)//2]
            I = I[:len(I)//2]
            
        if tafel_plot == True:
            V = V[100:len(V)//2]
            I = I[100:len(I)//2]
            I = np.log10(abs(I)) #gets log of absolute value of current
            V,I = I,V #swap axes
        return V,I

def smooth(y,box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y,box,mode='same')
    return y_smooth

def normalize_func(array):
    """

    Parameters
    ----------
    array : a NumPy array one wishes to normalize (make such that the max
    value = 1)

    Returns
    -------
    Returns the given array normalized such that the max value = 1

    """
    return array/max(abs(array))

def find_capacitance(I,V,capacitance_pot_range = [-0.2,0.1],reverse=None,scan_rate=0.5):
    """

    Parameters
    ----------
    I : ARRAY. The current array for which we want to find the capacitance
    V : ARRAY. The voltage array for which we want to find the capacitance
    capacitance_pot : list of 2 ints. The potential range over which we will
    determine the capacitance

    Returns
    -------
    capacitance
    
    This integrates current wrt time then finds capacitance this way
    """
    potential_window = capacitance_pot_range[1]-capacitance_pot_range[0]
    #convert potential window to volts if in millivolts
    if potential_window > 5.0:
        potential_window = potential_window/1000
        
    # print("len(v)/2)=",len(V)/2)
    first_half_V = V[0:int(len(V)/2)]
    closest_pot_min = V[0]
    for i in first_half_V:
        #print('Checking',i,'mv')
        if abs(i-capacitance_pot_range[0]) <= abs(closest_pot_min-capacitance_pot_range[0]):
            closest_pot_min = i
            
    closest_pot_max = V[0]
    for i in first_half_V: 
        #print('Checking',i,'mv')
        if abs(i-capacitance_pot_range[1]) <= abs(closest_pot_max-capacitance_pot_range[1]):
            closest_pot_max = i
    # print('Closest potential found=',closest_pot_min,closest_pot_max)
    #finds the indices of both values of potential which are closest to the desired
    index_min = int(np.where(V == closest_pot_min)[0][0])
    index_max = int(np.where(V == closest_pot_max)[0][0])
    I,T = convert_cv_to_time(V,I,scan_rate)
    area_forward = np.trapz(I[index_max:index_min],x=T[index_max:index_min])
    area_backward = np.trapz(I[-index_min:-index_max],x=T[-index_min:-index_max])
    area = area_backward-area_forward #the integral of current wrt time over given potential window
    print("Area=",area)
    capacitance = area / (potential_window)
    return capacitance


def plotter(folders=[],colors=[],
           saveas=None,
           title=None,
           double_scan=True,
           scan_labels = [],
           current_density=False,
           diameter=7e-5,
           linewidth=1.0,
           legend=True,
           normalize=False,
           custom_normalize = [],
           smoothen = False,
           capacitance_normalize = False,
           first_half_only = False,
        custom_legend = False,
        xlabel = None,
        ylabel = None,
           tafel_plot = False,
           reverse = False):
    """
    custom_normalize: list of lists (or similar iterable). Number of lists 
    must be equal to the number of folders, length of each internal list
    must be equal to the number of valid files in the folder it matches
    """

    fig, ax = plt.subplots(figsize=(8,4.5),dpi=100)
    print('plot')
    for i in range(len(folders)):
        folder = folders[i]
        file_names,data_files = gen_text_from_folder(folder)
        if len(custom_normalize) == len(folders):
            custom_normalize_list = list(custom_normalize[i])
        else: 
            custom_normalize_list = None
        V_list = []
        I_list = []
        for j in range(len(data_files)):
            data_file = data_files[j]
            file_name = file_names[j]
            # print('len of data files = ',len(file_names),'len custom normalize list = ',len(custom_normalize_list))
            if type(custom_normalize_list) == list:
                custom_normalization_value = custom_normalize_list[j]
            else:
                custom_normalization_value = None
            V, I = gen_array_from_text(data_file,file_name)
            V,I = array_modifier(V,I,
                    double_scan = double_scan,
                    reverse = reverse,
                    smoothen=smoothen,
                    normalize = normalize,
                    capacitance_normalize=capacitance_normalize,
                    custom_normalize = custom_normalization_value,
                    first_half_only=first_half_only,
                    tafel_plot = tafel_plot)
            ax.plot(V,I,color=colors[i],linewidth=linewidth)

    if tafel_plot == True:
        xlabel = "log(Current)"
        ylabel = "Potential (V)"
    ax.yaxis.set_minor_locator(AutoMinorLocator())
    ax.yaxis.set_ticks_position('both')
    ax.xaxis.set_minor_locator(AutoMinorLocator())
    ax.grid(True)
    ax.set_xlabel(xlabel)
    ax.set_ylabel(ylabel)
    if capacitance_normalize==True:
        ax.set_ylabel("Current, normalized by capacitance")
    if title!=None:
        if normalize == True:
            title = title + ", normalized by current"
        if capacitance_normalize == True:
            title = title + ", capacitance normalized"
        ax.set_title(title)
    
    if custom_legend == True:
        custom_lines = [Line2D([0], [0], color=colors[0], lw=4),
                Line2D([0], [0], color=colors[1], lw=4)]
        ax.legend(custom_lines, ['Substrate','Ni NPs'])
    if saveas!=None:
        if normalize == True:
            saveas += "_currentnorm"
        if capacitance_normalize==True:
            saveas += "_capacitancenorm"
        if first_half_only==True:
            saveas += "_linear"
        if tafel_plot == True:
            saveas += "_tafel"
        fig.savefig(saveas,bbox_inches='tight')
        print('saved')
    plt.show()
    
def get_droplet_sizes(filepath):
    """
    Generates NumPy array of the droplet sizes listed in a .csv
    file generated by ImageJ
    filepath: path to the desired .csv file
    """
    # with open(filepath) as file_name:
    #     array = np.loadtxt(file_name, delimiter=",")
    data = np.genfromtxt(filepath,skip_header=1,delimiter=',')
    data = np.hsplit(data,5)
    droplet_area = data[1].flatten()
    return droplet_area
    

In [38]:
#SECCM on Ni NPs AND Ni@C NPs using 0.1 M H2SO4
substrate_bare_folder = "/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni NPs (no carbon shell)/linescan_1_substrate"
dropcast_bare_folder = "/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni NPs (no carbon shell)/linescan_1_dropcast"
substrate_sputter_folder = "/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni@C NPs/linescan_1_substrate"
dropcast_sputter_folder = "/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni@C NPs/linescan_1_dropcast"

plotter(folders = [substrate_bare_folder,dropcast_bare_folder,substrate_sputter_folder,dropcast_sputter_folder],
       colors = ['m','c','r','b'],
       # first_half_only=True,
       # capacitance_normalize=True,
        normalize=True,
        # tafel_plot=True
       )

plot


In [17]:
#SECCM on just Ni NPs in 0.1M H2SO4
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni NPs (no carbon shell)/linescan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni NPs (no carbon shell)/linescan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni NPs (no carbon shell)/Line scan 1 plot"

plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,
            saveas=saveas,
            title = "HER SECCM on anC vs Ni NPs in 0.1 M H$_2$SO$_4$",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            # normalize=True,
            first_half_only=True,
            # capacitance_normalize=True,
            # tafel_plot=True
            )

plot
saved


In [47]:
#SECCM on Ni@C NPs in 0.1M H2SO4
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni@C NPs/linescan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni@C NPs/linescan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-28 HER SECCM on Ni NPs/Ni@C NPs/Line scan 1 plot"

plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,
            # saveas=saveas,
            title = "HER SECCM on anC vs Ni NPs in 0.1 M H$_2$SO$_4$",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            normalize=True,
            first_half_only=True,
            # capacitance_normalize=True,
            # tafel_plot=True
            )

plot
saved


In [40]:
#SECCM on Ni NPs at pH = 1.7
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-21 HER SECCM (acid) Ni NPs (look for oxidation)/linescan_2_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-21 HER SECCM (acid) Ni NPs (look for oxidation)/linescan_2_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-21 HER SECCM (acid) Ni NPs (look for oxidation)/Line scan 2 plot"

plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,
            # saveas=saveas,
            title = "HER SECCM on anC vs Ni NPs at pH = 1.7",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            first_half_only=True,
            # capacitance_normalize=True,
            # reverse = True,
            # tafel_plot=True
            )

plot


In [55]:
#SECCM on Ni@C NPs (4') in 0.1M H2SO4
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-20 HER SECCM (0.1M acid) Ni@C NPs/Pipette 1/Substrate and dropcast/linescan_8_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-20 HER SECCM (0.1M acid) Ni@C NPs/Pipette 1/Substrate and dropcast/linescan_8_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-20 HER SECCM (0.1M acid) Ni@C NPs/Pipette 1/Substrate and dropcast/Line scan 8 plots"

plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,linewidth=0.7,
            # saveas=saveas,
            title = "HER SECCM on anC vs Ni@C NPs in 0.1 M H$_2$SO$_4$",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            first_half_only=True,
            # capacitance_normalize=True,
            # reverse = True,
            tafel_plot=True
            )

plot


In [47]:
#SECCM on Ni@C NPs (4') at pH = 1.7
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-13 HER SECCM (acid) Ni@C NPs/Pipette 1/Substrate and dropcast/linescan_4_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-13 HER SECCM (acid) Ni@C NPs/Pipette 1/Substrate and dropcast/linescan_4_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-13 HER SECCM (acid) Ni@C NPs/Pipette 1/Substrate and dropcast/linescan_4_comparison"

plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,linewidth=0.7,
            # saveas=saveas,
            title = "HER SECCM on anC vs anC/Ni NPs at pH = 1.7",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            # first_half_only=True,
            capacitance_normalize=True,
            )

plot
90 -0.39762 30 0.00082036
90 -0.39762 30 0.00082036
90 -0.39762 30 0.00082036
90 -0.39762 30 0.00082036


In [50]:
#SECCM on Ni NPs at pH = 2
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-04 SECCM on Ni NPs (acid)/Pipette 4/Substrate and drop-cast/scan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-04 SECCM on Ni NPs (acid)/Pipette 4/Substrate and drop-cast/scan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-04 SECCM on Ni NPs (acid)/Pipette 4/Substrate and drop-cast/scan_1_tafel"
# filepath_substrate_sizes = "/home/larry/Documents/Capstone Project/Data/21-10-04 SECCM on Ni NPs (acid)/Pipette 4/Substrate and drop-cast/scan_1_substrate_dropletsize.csv"
# filepath_dropcast_sizes = "/home/larry/Documents/Capstone Project/Data/21-10-04 SECCM on Ni NPs (acid)/Pipette 4/Substrate and drop-cast/scan_1_dropcast_dropletsize.csv"

# substrate_custom_normalize = get_droplet_sizes(filepath_substrate_sizes)
# dropcast_custom_normalize = get_droplet_sizes(filepath_substrate_sizes)
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,linewidth=0.7,
            # saveas=saveas,
            title = "HER SECCM on anC vs anC/Ni NPs at pH = 1.7",
            xlabel="log(Current)",ylabel="Potential vs PdH (V)",
            # custom_normalize = [substrate_custom_normalize,dropcast_custom_normalize],
            first_half_only=True,
            # capacitance_normalize=True,
            tafel_plot = True,
            )

plot
saved


In [49]:
#SECCM on Cu NWs in 0.1M H2SO4
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Cu NWs (0.1M acid)/linescan_3_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Cu NWs (0.1M acid)/linescan_3_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Cu NWs (0.1M acid)/linescan3_comparison"
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
            custom_legend=True,linewidth=0.7,
            # saveas=saveas,
            title = "HER SECCM on anC vs anC/Cu NWs w/ 0.1 M H$_2$SO$_4$",
            xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
            # first_half_only=True,
            # capacitance_normalize=True,
           )

plot


In [55]:
##SECCM on Ni NPs in 0.1M H2SO4
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Ni NPs (0.1M acid)/Pipette 2/linescan_2_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Ni NPs (0.1M acid)/Pipette 2/linescan_2_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-07 SECCM on Ni NPs (0.1M acid)/Pipette 2/linescan2_comparison"
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
        custom_legend=True,linewidth=0.7,
        # saveas=saveas,
        title = "HER SECCM on anC vs anC/Ni NPs w/ 0.1 M H$_2$SO$_4$",
        xlabel="Potential vs PdH (V)",ylabel="Current (nA)",
        # first_half_only=True,
        # capacitance_normalize=True,
       )


plot


In [5]:
#SECCM on Ni NPs at pH=12
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-06 SECCM on Ni NPs (base)/Pipette 1/Substrate and dropcast/scan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-10-06 SECCM on Ni NPs (base)/Pipette 1/Substrate and dropcast/scan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-10-06 SECCM on Ni NPs (base)/Pipette 1/Substrate and dropcast/scan_1_comparison_capacitancenorm"
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
       custom_legend=True,linewidth=0.7,
       saveas=saveas,
       title = "HER SECCM on anC vs anC/Ni NPs at pH=12",
       xlabel="Potential vs Pt (V)",ylabel="Current (nA)",
       # first_half_only=True,
       capacitance_normalize=True,
       )

plot
saved


In [8]:
#SECCM on Ni NPs at pH=7
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_comparison_capacitancenorm"
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
       custom_legend=True,linewidth=0.7,
       saveas=saveas,
       title = "HER SECCM on anC vs anC/Ni NPs at pH=7",
       xlabel="Potential vs Ag/AgCl (V)",ylabel="Current, capacitance normalized",
       # first_half_only=True,
       capacitance_normalize=True,
       )

plot
saved


In [9]:
#SECCM on Ni NPs at pH=7
substrate_folder = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_substrate" 
dropcast_folder = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_dropcast"
saveas = r"/home/larry/Documents/Capstone Project/Data/21-09-28 SECCM on Ni NPs/Medium pipette 2/scan_1_comparison_capacitancenorm"
plotter(folders=[substrate_folder,dropcast_folder],colors=['r','b'],
       custom_legend=True,linewidth=0.7,
       saveas=saveas,
       title = "HER SECCM on anC vs anC/Ni NPs at pH=7",
       xlabel="Potential vs Ag/AgCl (V)",ylabel="Current, capacitance normalized",
       # first_half_only=True,
       capacitance_normalize=True,
       )

plot
saved


plot
saved
